In [1]:
import pandas as pd
from datetime import datetime, timedelta

# Sample data creation
data = {
    'product': ['Mountain', 'Road', 'Hybrid', 'Mountain', 'Road', 'Hybrid', 'Mountain', 'Road', 'Hybrid'],
    'quantity': [120, 150, 130, 140, 160, 135, 150, 165, 140],
    'date_ordered': [datetime(2023, 1, 1) + timedelta(days=i) for i in range(9)],
    'price': [500, 800, 650, 520, 820, 660, 530, 830, 670]
}

df = pd.DataFrame(data)
print(df)

    product  quantity date_ordered  price
0  Mountain       120   2023-01-01    500
1      Road       150   2023-01-02    800
2    Hybrid       130   2023-01-03    650
3  Mountain       140   2023-01-04    520
4      Road       160   2023-01-05    820
5    Hybrid       135   2023-01-06    660
6  Mountain       150   2023-01-07    530
7      Road       165   2023-01-08    830
8    Hybrid       140   2023-01-09    670


In [4]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from keras.utils import to_categorical
import numpy as np

# Configuring preprocessing pipeline
column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(), ['product']),
    ('scaler', StandardScaler(), ['quantity', 'price'])
])

# Fitting and transforming the data
processed_data = column_transformer.fit_transform(df)

# Creating sequences
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        end_ix = i + n_steps
        if end_ix > len(data) - 1:
            break
        seq_x, seq_y = data[i:end_ix], data[end_ix, 1]  # Assuming the next quantity to be predicted
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

n_steps = 3  # Using last 3 days to predict the next day
X, y = create_sequences(processed_data, n_steps)




In [5]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Input

# Define the model using an Input layer
model = Sequential([
    Input(shape=(n_steps, X.shape[2])),  # Clearly define the input shape
    SimpleRNN(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Now you can proceed with model training
# model.fit(X, y, epochs=30, validation_split=0.1)


In [6]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Input

# Creating a sample dataset
np.random.seed(42)
dates = [datetime(2023, 1, 1) + timedelta(days=i) for i in range(100)]
products = np.random.choice(['Mountain', 'Road', 'Hybrid'], size=100)
quantities = np.random.randint(100, 200, size=100)
prices = np.random.normal(1, 0.1, size=100) * np.where(products == 'Mountain', 500, np.where(products == 'Road', 800, 650))

df = pd.DataFrame({
    'date_ordered': dates,
    'product': products,
    'quantity': quantities,
    'price': prices
})

# Preprocessing
column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(), ['product']),
    ('scaler', StandardScaler(), ['quantity', 'price'])
])

data = column_transformer.fit_transform(df)
n_steps = 7
n_features = data.shape[1]

# Create sequences
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        end_ix = i + n_steps
        seq_x, seq_y = data[i:end_ix], data[end_ix - 1, 1]  # Use quantity of the last timestep as label
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

X, y = create_sequences(data, n_steps)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model definition
model = Sequential([
    Input(shape=(n_steps, n_features)),
    SimpleRNN(50, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Fit the model
history = model.fit(X_train, y_train, epochs=30, validation_split=0.1)

# Model evaluation
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")


Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 167ms/step - loss: 0.1721 - mae: 0.3238 - val_loss: 0.2207 - val_mae: 0.4153
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.1495 - mae: 0.3058 - val_loss: 0.1895 - val_mae: 0.3826
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.1262 - mae: 0.2710 - val_loss: 0.1657 - val_mae: 0.3568
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1018 - mae: 0.2418 - val_loss: 0.1435 - val_mae: 0.3284
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0891 - mae: 0.2241 - val_loss: 0.1230 - val_mae: 0.3013
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0776 - mae: 0.2110 - val_loss: 0.1091 - val_mae: 0.2840
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0630 - mae: 0.1923 - val_loss: 0.0960 - val_mae: 0.2656
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0554 - mae: 0.1820 - val_loss: 0.0818 - val_mae: 0.2468
Epoch 9/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0430 - mae: 0.1579 -